# Setup

In [ ]:
import pandapower.networks as pnet
import pandas as pd
import numpy as np
import scipy as sp
import cvxpy as cp
import seaborn as sns
import mlflow

from scipy import sparse

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, '..')

from src.models.matrix_operations import make_real_vector, vectorize_matrix
from src.simulation.noise import add_polar_noise_to_measurement
from src.models.regression import ComplexRegression, ComplexLasso
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import error_metrics, fro_error
from src.models.noise_transformation import average_true_noise_covariance

# Network simulation

In [ ]:
mlflow.set_experiment('Full network with cartesian noise')

In [ ]:
net = pnet.create_kerber_landnetz_freileitung_2()
steps = 400
load_cv = 0.2
v_noise_sds = np.array([0.02, 0.08, 0.01, 0.07, 0.05, 0.03, 0.04, 0.02, 0.06]) * 1e-3
i_noise_sds = np.array([0.03, 0.04, 0.01, 0.02, 0.04, 0.07, 0.02, 0.04, 0.06]) * 1e-4

In [ ]:
np.random.seed(11)
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, steps)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)
controlled_net.bus

In [ ]:
voltage = voltage[:, 1:]
y_bus = y_bus[1:, 1:]
current = np.array(voltage @ y_bus)
nodes = voltage.shape[1]

noisy_voltage = voltage.copy() + np.random.normal(0, v_noise_sds, (steps, nodes)) + 1j*np.random.normal(0, v_noise_sds, (steps, nodes))
noisy_current = current.copy() + np.random.normal(0, i_noise_sds, (steps, nodes)) + 1j*np.random.normal(0, i_noise_sds, (steps, nodes))

voltage_error, current_error = noisy_voltage - voltage, noisy_current - current

In [ ]:
np.linalg.svd(voltage, compute_uv=False)

In [ ]:
np.mean(np.abs(current), 0)

In [ ]:
mlflow_params = {
    'nodes': nodes,
    'steps': steps, 
    'load_cv': load_cv, 
    'v_noise_sds': v_noise_sds, 
    'i_noise_sds': i_noise_sds
}

# OLS Identification

In [ ]:
with mlflow.start_run(run_name='OLS'):
    ols = ComplexRegression()
    ols.fit(noisy_voltage, noisy_current)
    y_ols = ols.fitted_admittance_matrix
    ols_metrics = error_metrics(y_bus, y_ols)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(ols_metrics.__dict__)
ols_metrics

# Lasso Identification

In [ ]:
with mlflow.start_run(run_name='Lasso'):
    lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-12, -1, 40), solver=cp.GUROBI)
    lasso.fit(noisy_voltage, noisy_current)
    y_lasso = lasso.fitted_admittance_matrix
    lasso_metrics = error_metrics(y_bus, y_lasso)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(lasso_metrics.__dict__)
best_lambda = lasso.best_trial.hyperparameters['lambda']
lasso_metrics

In [ ]:
best_lambda

# TLS Identification

In [ ]:
with mlflow.start_run(run_name='TLS'):
    tls = TotalLeastSquares()
    tls.fit(noisy_voltage, noisy_current)
    y_tls = tls.fitted_admittance_matrix
    tls_metrics = error_metrics(y_bus, y_tls)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(tls_metrics.__dict__)
tls_metrics

# L1 Regularized TLS

In [ ]:
with mlflow.start_run(run_name='S-TLS no covariance'):
    max_iterations = 50
    abs_tol = 10e-12
    rel_tol = 10e-12
    solver = cp.GUROBI
    use_cov_matrix = False
    
    sparse_tls = SparseTotalLeastSquare(lambda_value=best_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
    sparse_tls.fit(noisy_voltage, noisy_current)
    
    y_sparse_tls = sparse_tls.fitted_admittance_matrix
    sparse_tls_metrics = error_metrics(y_bus, y_sparse_tls)
    
    sparse_tls_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_tls.iterations])
    sparse_tls_targets = pd.Series([i.target_function for i in sparse_tls.iterations])
    
    mlflow.log_param('lambda', best_lambda)
    mlflow.log_param('max_iterations', max_iterations)
    mlflow.log_param('abs_tol', abs_tol)
    mlflow.log_param('rel_tol', rel_tol)
    mlflow.log_param('solver', solver)
    mlflow.log_param('use_cov_matrix', use_cov_matrix)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(sparse_tls_metrics.__dict__)

    for i in range(len(sparse_tls_errors)):
        mlflow.log_metric('fro_error_evo', value=sparse_tls_errors[i], step=i)
        mlflow.log_metric('opt_cost_evo', value=sparse_tls_targets[i], step=i)

In [ ]:
sparse_tls_errors.plot()

In [ ]:
sparse_tls_targets.plot()

In [ ]:
def create_inv_cov(sds, samples):
    variances = np.array(sds)**2
    variance_diag = np.concatenate([np.repeat(variances, samples), np.repeat(variances, samples)])
    cov_matrix = sp.sparse.diags(variance_diag, format='csc')
    inv_cov_matrix = sp.sparse.linalg.inv(cov_matrix)
    return inv_cov_matrix

In [ ]:
with mlflow.start_run(run_name='S-TLS with covariance'):
    max_iterations = 50
    abs_tol = 10e-12
    rel_tol = 10e-12
    solver = cp.GUROBI
    use_cov_matrix = True
    
    inv_sigma_current = create_inv_cov(i_noise_sds, steps)
    inv_sigma_voltage = create_inv_cov(v_noise_sds, steps)
    
    sparse_tls_cov = SparseTotalLeastSquare(lambda_value=best_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
    sparse_tls_cov.fit(noisy_voltage, noisy_current, inv_sigma_voltage, inv_sigma_current)
    
    y_sparse_tls_cov = sparse_tls_cov.fitted_admittance_matrix
    sparse_tls_cov_metrics = error_metrics(y_bus, y_sparse_tls_cov)
    
    sparse_tls_cov_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_tls_cov.iterations])
    sparse_tls_cov_targets = pd.Series([i.target_function for i in sparse_tls_cov.iterations])
    
    mlflow.log_param('lambda', best_lambda)
    mlflow.log_param('max_iterations', max_iterations)
    mlflow.log_param('abs_tol', abs_tol)
    mlflow.log_param('rel_tol', rel_tol)
    mlflow.log_param('solver', solver)
    mlflow.log_param('use_cov_matrix', use_cov_matrix)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(sparse_tls_cov_metrics.__dict__)

    for i in range(len(sparse_tls_cov_errors)):
        mlflow.log_metric('fro_error_evo', value=sparse_tls_cov_errors[i], step=i)
        mlflow.log_metric('opt_cost_evo', value=sparse_tls_cov_targets[i], step=i)

In [ ]:
sparse_tls_cov_errors.plot()

In [ ]:
sparse_tls_cov_targets.plot()

# Result analysis

In [ ]:
sns.heatmap(np.abs(y_bus));

In [ ]:
sns.heatmap(np.abs(y_bus - y_ols));

In [ ]:
sns.heatmap(np.abs(y_bus - y_lasso));

In [ ]:
sns.heatmap(np.abs(y_bus - y_tls));

In [ ]:
sns.heatmap(np.abs(y_bus - y_sparse_tls));

In [ ]:
sns.heatmap(np.abs(y_bus - y_sparse_tls_cov));